## 🔮 बोल  भिडू ! 

In [89]:
import cohere
import streamlit as st
co = cohere.Client(st.secrets.COHERE_API_KEY)


### 1. Loading document

In [90]:
with open("C:/Users/mayur dabade/Desktop/Projects/marathi RAG/data/maharaj.txt", encoding='utf-8') as f:
    text = f.read()

print(f"The text has roughly {len(text.split())} words.")

The text has roughly 1030 words.


### 2. Splitting doc into chunks

In [91]:
# For chunking let's use langchain to help us split the text
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [92]:
# Create basic configurations to chunk the text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

# Split the text into chunks with some overlap
chunks_ = text_splitter.create_documents([text])
chunks = [c.page_content for c in chunks_]
print(f"The text has been broken down in {len(chunks)} chunks.")

The text has been broken down in 15 chunks.


### 3. Embed every text chunk


In [93]:
# Because the texts being embedded are the chunks we are searching over, we set the input type as search_doc
model="embed-multilingual-v3.0"
response = co.embed(
    texts= chunks,
    model=model,
    input_type="search_document",
    embedding_types=['float']
)
embeddings = response.embeddings.float
print(f"We just computed {len(embeddings)} embeddings.")

We just computed 15 embeddings.


### 4. Store the embeddings in a vector database

We use the simplest vector database ever: a python dictionary using `np.array()`.

In [94]:
import numpy as np
vector_database = {i: np.array(embedding) for i, embedding in enumerate(embeddings)}
# { 0: array([...]), 1: array([...]), 2: array([...]), ..., 10: array([...]) }

### 5. Given a user query, retrieve the relevant chunks from the vector database


### Define the user question

In [95]:
# query = "सगळ्यात जास्त शेअर असणाऱ्या भारतातील कंपनी?"
# query = "रिलायन्स इंडस्ट्रीज आणि डिजनी स्टार यांच्या मर्जरमुळे भारतातील एंटरटेनमेंट इंडस्ट्रीमध्ये कोणते बदल होण्याची शक्यता आहे?"
query="सीसीआयने या मर्जरला काही अटींसह मंजुरी का दिली? या अटींचा उद्देश काय असू शकतो?"
# query = "या मर्जरमुळे जागतिक पातळीवर भारताची प्रतिमा कशी बदलू शकते?"


### 6. Embed the user question


In [96]:
# Because the text being embedded is the search query, we set the input type as search_query
response = co.embed(
    texts=[query],
    model=model,
    input_type="search_query",
    embedding_types=['float']
)
query_embedding = response.embeddings.float[0]
# print("query_embedding: ", query_embedding)

### 7. Retrieve the most relevant chunks from the vector database

We use cosine similarity to find the most similar chunks

In [97]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Calculate similarity between the user question & each chunk
similarities = [cosine_similarity(query_embedding, chunk) for chunk in embeddings]
print("similarity scores: ", similarities)

# Get indices of the top 10 most similar chunks
sorted_indices = np.argsort(similarities)[::-1]

# Keep only the top 10 indices
top_indices = sorted_indices[:5]
print("Here are the indices of the top 5 chunks after retrieval: ", top_indices)

# Retrieve the top 10 most similar chunks
top_chunks_after_retrieval = [chunks[i] for i in top_indices]
print("Here are the top 5 chunks after retrieval: \n")
for t in top_chunks_after_retrieval:
    print("\n== " + t)


similarity scores:  [0.39042216797176, 0.535859393001034, 0.5209220639794689, 0.5239729444354693, 0.580017305183357, 0.6314604602118782, 0.4650150473179502, 0.5710279982043929, 0.47029629840894555, 0.4687914650110989, 0.45571845249419357, 0.47513857655440933, 0.597099167130034, 0.541507375359911, 0.38312835166689446]
Here are the indices of the top 5 chunks after retrieval:  [ 5 12  4  7 13]
Here are the top 5 chunks after retrieval: 


== संस्था भारतीय बाजारपेठेतील स्पर्धा टिकवून ठेवणं तिला प्रोत्साहन देण्याचं काम करते रिलायन्स आणि डिजनीच्या डीलमुळं भारतीय एंटरटेनमेंट मार्केटमधली स्पर्धा नष्ट होण्याची भीती व्यक्त केली जात होती त्यामुळे सीसीआय या डील साठी आधी तयार नव्हती फेब्रुवारी पासून सीसीआय न्या या मर्जरला परवानगी देण्याचा निर्णय राखून ठेवला होता मात्र आता काही अटींसह 28 ऑगस्टला सीसीआयन या डीलला परवानगी दिली आहे या कंपनीला सीसीआय न दिलेल्या परवानगी नंतर आता डिजनीचे 80 आणि वायकॉम 18 चे 40 असे एकूण 120 टीव्ही चॅनल्स तयार होणाऱ्या नवीन

== परिणाम जाहिरात क्षेत्रावर होऊ शकतो तिथल्या छो

### 8. designing final answer

In [98]:
# Prepare the context for the template
context = "\n".join(top_chunks_after_retrieval)

# Template for the answer
template = f"""Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. answer should be in marathi
 
{context}
Question: {query}
Helpful Answer:"""

# print(template)

In [99]:
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass(st.secrets.GOOGLE_API_KEY)

llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke(template)
print(result.content)

या प्रश्नाचे उत्तर दिलेल्या मजकूरात दिलेले नाही.


In [100]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=0, groq_api_key=st.secrets.GROQ_API_KEY, model_name="llama3-8b-8192")


system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
a = chain.invoke({"text": template})
print(a.content)

सीसीआयने या मर्जरला 28 ऑगस्टला काही अटींसह मंजुरी दिली आहे. ह्या अटींचा उद्देश म्हणजे स्पर्धा टिकवून ठेवणं, क्रिकेटचे कोट्यावधी चाहते आणि प्रेक्षकांसमोर दुसरा कोणताही पर्याय शिल्लक नसल्यामुळे ही कंपनी आपल्या सबस्क्रिप्शनचे दर वाढवण्याची शक्यताही आहे.
